In [11]:
from scipy import stats
from numpy import log, exp, sqrt
import numpy as np

In [2]:
def call_option_price(S, E, T, rf, sigma):
    # calculate d1 and d2 params
    # t=0 => T-t=T
    d1 = (log(S/E)+(rf+sigma*sigma/2)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)

    return S*stats.norm.cdf(d1)-E*exp(-rf*T)*stats.norm.cdf(d2)

In [3]:
def put_option_price(S, E, T, rf, sigma):
    # calculate d1 and d2 params
    # t=0 => T-t=T
    d1 = (log(S/E)+(rf+sigma*sigma/2)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)

    return -S*stats.norm.cdf(-d1)+E*exp(-rf*T)*stats.norm.cdf(-d2)

In [26]:
class OptionPricing:
    def __init__(self, S0, E, T, rf, sigma, iterations):
        self.S0 = S0
        self.E = E
        self.T = T
        self.rf = rf
        self.sigma = sigma
        self.iterations = iterations

    def call_option_simulation(self):
        # payoff function is max(0, S-E) for call option
        option_data = np.zeros([self.iterations, 2])

        rand = np.random.normal(0, 1, [1, self.iterations])

        # equation for the S(t) stock price at T
        stock_price = self.S0 * np.exp(self.T * (self.rf - 0.5 * self.sigma **2) + self.sigma * np.sqrt(self.T) * rand)

        # calculate S-E
        option_data[:, 1] = stock_price - self.E

        # average for MC
        average = np.sum(np.amax(option_data, axis=1)) / float(self.iterations)

        # discount factor for present value
        return np.exp(-1.0*self.rf*self.T)*average
    
    def put_option_simulation(self):
        # payoff function is max(0, E-S) for call option
        option_data = np.zeros([self.iterations, 2])

        rand = np.random.normal(0, 1, [1, self.iterations])

        # equation for the S(t) stock price at T
        stock_price = self.S0 * np.exp(self.T * (self.rf - 0.5 * self.sigma **2) + self.sigma * np.sqrt(self.T) * rand)

        # calculate E-S
        option_data[:, 1] = self.E - stock_price

        # average for MC
        average = np.sum(np.amax(option_data, axis=1)) / float(self.iterations)

        # discount factor for present value
        return np.exp(-1.0*self.rf*self.T)*average

In [29]:
def main():
    S0 = 100
    E = 100
    T = 1
    rf = 0.05
    sigma = 0.2

    call = call_option_price(S0, E, T, rf, sigma)
    print(call)

    put = put_option_price(S0, E, T, rf, sigma)
    print(put)

    model = OptionPricing(100, 100, 1, 0.05, 0.2, 100000)
    print("Value of call option is $%.2f" % model.call_option_simulation())
    print("Value of put option is $%.2f" % model.put_option_simulation())

In [30]:
if __name__ == "__main__":
    main()

10.450583572185565
5.573526022256971
Value of call option is $10.43
Value of put option is $5.60
